In [3]:
import cv2
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from skimage.feature import hog
import numpy as np
import csv


In [1]:
def extract_features(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        return None
    image_resized = cv2.resize(image, (128, 128))
    features, _ = hog(image_resized, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys', visualize=True)
    return features


In [5]:
def extract_and_save_features(folder, label, csv_path):
    if not os.path.exists(folder):
        print(f"Folder '{folder}' not found.")
        return False
    
    features_list = []
    
    for filename in os.listdir(folder):
        image_path = os.path.join(folder, filename)
        features = extract_features(image_path)
        if features is not None:
            features_list.append(features.tolist() + [label])
    
    df = pd.DataFrame(features_list)
    if not os.path.exists(csv_path):
        df.to_csv(csv_path, index=False, mode='w')
    else:
        df.to_csv(csv_path, mode='a', header=False, index=False)
    
    return True


In [7]:
genuine_folder = 'D:/Signature Verification/genuine_signatures'
test_folder = 'D:/Signature Verification/test_signatures'
train_csv = 'D:/Signature Verification/features.csv'

# Extract features for genuine signatures
genuine_exists = extract_and_save_features(genuine_folder, 1, train_csv)

# Extract features for test signatures
test_exists = extract_and_save_features(test_folder, 0, train_csv)

# Ensure both folders were processed
if not genuine_exists and not test_exists:
    print("No 'genuine_signatures' or 'test_signatures' folder found.")
elif not genuine_exists:
    print("No 'genuine_signatures' folder found.")
elif not test_exists:
    print("No 'test_signatures' folder found.")
else:
    print("Feature extraction completed.")


Feature extraction completed.


In [9]:
def train_model(csv_file):
    if not os.path.exists(csv_file):
        print(f"No feature CSV file found at {csv_file}. Please extract features first.")
        return None
    
    df = pd.read_csv(csv_file)
    X = df.iloc[:, :-1].values
    y = df.iloc[:, -1].values
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    clf = SVC(kernel='linear')
    clf.fit(X_train, y_train)
    
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    
    print(f"Model training completed with an accuracy of {acc * 100:.2f}%")
    return clf


In [11]:
model = train_model(train_csv)


Model training completed with an accuracy of 100.00%


In [13]:
def compare_signature(image_path, model, csv_file):
    if model is None:
        print("No trained model found. Please train the model first.")
        return
    
    features = extract_features(image_path)
    if features is None:
        print("Unable to extract features from the image.")
        return
    
    features = np.array(features).reshape(1, -1)
    prediction = model.predict(features)[0]
    
    if prediction == 1:
        print("The signature is genuine.")
    else:
        print("The signature is forged.")


In [ ]:
def compare_signature(image_path, model, csv_file):
    if model is None:
        print("No trained model found. Please train the model first.")
        return
    
    features = extract_features(image_path)
    if features is None:
        print("Unable to extract features from the image.")
        return
    
    features = np.array(features).reshape(1, -1)
    prediction = model.predict(features)[0]
    
    if prediction == 1:
        print("The signature is genuine.")
    else:
        print("The signature is forged.")

# Ask user to provide the path to the new signature image
new_signature_path = input("Enter the path to the new signature image for comparison: ")

# Remove any surrounding double quotes
new_signature_path = new_signature_path.strip('"')

compare_signature(new_signature_path, model, train_csv)
